In [ ]:
%pip install openai==0.28.1 --quiet 
%pip install langchain==0.0.320 --quiet
%pip install chromadb==0.4.14 --quiet
%pip install tiktoken==0.5.1--quiet

In [ ]:
# OpenAI API Key beállítása
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

### Paraméterek

In [ ]:
# használt erőforrások mappája
res_folder = "res/in_use/"
    
# splitting paraméterek
chunk_size = 500
chunk_overlap = 50

# vector store paraméterek
persist_directory = "res/chroma/"
search_type = "mmr"
search_k = 5
search_fetch_k = 8
lambda_mult = 0.6

# memória
memory_k = 3

# ChatGPT paraméterek
temperature = 0.4
max_tokens = 500
model_id = "gpt-3.5-turbo"

## Erőforrásfájlok betöltése

Ha már egyszer megtettük és van mentett vektor adatbázis, akkor nem kell újra futtatni.
### CSV fájlok (nagyrészt kérdések) betöltése

In [ ]:
from langchain.document_loaders import CSVLoader, DirectoryLoader

directory_loader = DirectoryLoader(res_folder, glob="*.csv", use_multithreading=True, loader_cls=CSVLoader, loader_kwargs={"encoding": "utf-8"})
csv_data = directory_loader.load()
print(len(csv_data))
#csv_data


### Szöveges fájlok betöltése, majd feldarabolása

In [ ]:
from langchain.document_loaders import DirectoryLoader, TextLoader

directory_loader = DirectoryLoader(res_folder, glob="*.txt", use_multithreading=True, loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"})
text_data = directory_loader.load()
text_data

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
    is_separator_regex=True,
    separators=["\n\s*\n", "\n\s*", "\n"]
)

split_text_data = text_splitter.split_documents(text_data)
print(len(split_text_data))
#split_text_data

### VectorStore inicializálása Chroma-val

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma

# Embedding betöltése
oai_embedding = OpenAIEmbeddings()

In [ ]:
# Elég egyszer futtatni, ha nem változtatunk az adatokon, mert lementi a vektoradatbázist.
# Változás esetén törölni kell a res/chroma maappát, majd újra kell futtatni.
combined_data = []
combined_data.extend(split_text_data)
combined_data.extend(csv_data)

In [ ]:
oai_vectordb = Chroma.from_documents(
    documents=combined_data,
    embedding=oai_embedding,
    persist_directory=persist_directory
)

oai_vectordb.persist()
vectordb = oai_vectordb

Ha le van már mentve, ezt kell használni az előző cella helyett:

In [ ]:
oai_vectordb = Chroma(persist_directory=persist_directory, embedding_function=oai_embedding)
vectordb = oai_vectordb

In [ ]:
# MMR teszt
question = "Mennyit kell dolgozni?"
search_result = vectordb.max_marginal_relevance_search(question,k = search_k, fetch_k = search_fetch_k, lambda_mult = lambda_mult)
print(search_result)
print(vectordb.similarity_search_with_relevance_scores(question))

### Memória config a chat historyhoz

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k = memory_k, memory_key="chat_history", return_messages=True)

## Chatbot OpenAI-jal

### Alap ConversationalRetrievalChain használatával

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI(model_name= model_id, temperature = temperature, max_tokens = max_tokens)
chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever = vectordb.as_retriever(
        searh_type = search_type,
        search_kwargs = {
            "k": search_k,
            "fetch_k": search_fetch_k,
            "lambda_mult": lambda_mult
        }
    ),
    verbose = True,
    memory = memory
)

### Saját prompttal

Az előző cella helyett lehet ezt használni, ha nem a beépített dolgot szeretnénk használni, jobb eredményekért:

In [ ]:
from langchain.chains import StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI


llm = ChatOpenAI(model_name= model_id, temperature = temperature, max_tokens = max_tokens)

Előzmények alappján konkrét, önálló kérdést generálunk:

In [ ]:

template = """A chat előzményekből és a következő inputból alakíts ki egy önmagában is helytálló kérdést, ha az input értelmezéséhez fontos kontextust tartalmaz!
Ha az input nem kérdés vagy nem kapcsolódik már az előzményekhez, akkor csak add vissza az inputot!

Chat előzmények:
{chat_history}

Következő input: {question}

Átalakított kérdés:"""
question_generator_prompt = PromptTemplate.from_template(template)
question_generator_chain = LLMChain(
    llm=llm,
    prompt=question_generator_prompt,
    verbose=True
)


Kontextus alapján válasz generálása:

In [ ]:
qa_template = """A BME VIK szakmai gyakorlat kérdéseire válaszoló chatbot vagy. A kérdésekre magyarul válaszolj!
Használd az alábbi dokumentumrészleteket forrásként a felhasználó kérdésének megválaszolásához!
Ha azokból nem tudsz megadni releváns választ, akkor válaszold azt, hogy "Sajnos erre nem tudok válaszolni, kérdezz mást a BME VIK szakmai gyakorlattal kapcsolatban"

DOKUMENTUMRÉSZLETEK:
{context}

ÚJ INPUT: {question}

Válaszolj a kérdésre!"""

qa_prompt = PromptTemplate.from_template(qa_template)
llm_chain = LLMChain(
    llm=llm,
    prompt=qa_prompt,
    verbose=True
)
combine_docs_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    verbose=True
)


In [ ]:
memory.clear()
chain = ConversationalRetrievalChain(
    combine_docs_chain=combine_docs_chain,
    retriever = vectordb.as_retriever(
        searh_type = search_type,
        search_kwargs = {
            "k": search_k,
            "fetch_k": search_fetch_k,
            "lambda_mult": lambda_mult
        }
    ),
    question_generator=question_generator_chain,
    memory=memory,
    #return_generated_question=True,
    rephrase_question=True,
    verbose=True
)

## Gradio UI a chatbothoz

In [ ]:
%pip install gradio==3.47.1 --quiet

In [ ]:
def qa(message, history) -> str:
    return chain({"question": message})["answer"]

Interfész létrehozása, indítása:

In [ ]:
import gradio as gr

chat_ui = gr.ChatInterface(qa, title = "Lacibot", description="Kérdezz a VIK-es szakmai gyakorlatról!", undo_btn=None)
chat_ui.launch() 

## Tesztelés


In [ ]:
memory.buffer

In [ ]:
memory.clear()

In [ ]:
# egyedüli kérdés
question = "Hogy hívják a tárgyfelelőst?"
chain({"question": question})

In [ ]:
%pip install pandas==2.1.1 --quiet
%pip install matplotlib==3.8.0 --quiet


#### Teszt kérdések betöltése

In [ ]:
import pandas as pd

test_questions = pd.read_csv("testing/test_questions.csv")
print(len(test_questions))
#test_questions

In [ ]:
def test_vectordb(vectordb, embedding_name: str) -> pd.DataFrame:
    df = pd.DataFrame(columns=['Question', 'Document', 'Score'])

    for question in test_questions['Question']:
        print(question)
        search_result = vectordb.similarity_search_with_relevance_scores(question, k = search_k)
        for res in search_result:
            df.loc[len(df)] = [question, res[0], res[1]]
    
    # elmentés
    df.to_csv(f"testing/embedding/{embedding_name}_k{search_k}_size{chunk_size}.csv", index=False)

    return df

#### Különböző embeddingekkel való tesztelés

Használatához a fájl elején lévő adatok beolvasására van szükség.

In [ ]:
%pip install InstructorEmbedding --quiet
%pip install sentence-transformers --quiet

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.chroma import Chroma

sentran_embedding = HuggingFaceEmbeddings(model_name= "sentence-transformers/distiluse-base-multilingual-cased-v2",model_kwargs={"device": "cpu"})
sentran_vectordb = Chroma.from_documents(
    documents=combined_data,
    embedding=sentran_embedding,
    persist_directory=persist_directory + "sentran/"
)


In [ ]:

from langchain.embeddings import HuggingFaceInstructEmbeddings

instr_embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-base",
    model_kwargs = {"device" : "cpu"},
    query_instruction="Represent the query for retrieval: "
)
instr_vectordb = Chroma.from_documents(
    documents=combined_data,
    embedding=instr_embeddings,
    persist_directory=persist_directory + "instr/"
)

In [ ]:
sentran_df = test_vectordb(sentran_vectordb, "sentence_tran")
instr_df = test_vectordb(instr_vectordb, "instructor")

In [ ]:
openai_df = test_vectordb(oai_vectordb, "openai")

#### Gráfok vectordb teszthez

In [ ]:
import matplotlib.pyplot as plt

def make_diagram(df, diagram_name: str):
    grouped = df.groupby('Question')

    fig, ax = plt.subplots()

    # Iterate over each group
    for i, (name, group) in enumerate(grouped, start=1):
        group = group.reset_index(drop=True)
        ax.plot(group.index+1, group['Score'], label=f"Group {i}")

    ax.legend(fontsize=8, loc='upper right')
    plt.xlabel('Index')
    plt.xlim(0.75, search_k+0.25)
    plt.xticks(range(1, search_k+1))

    plt.ylabel('Score')
    plt.ylim(min(df['Score']) - 0.005, max(df['Score']) + 0.005)

    plt.title(diagram_name)
    plt.show()

In [ ]:
make_diagram(sentran_df, "Sentence Transformer")
make_diagram(instr_df, "Instructor")
make_diagram(openai_df, "OpenAI")

### Full tesztek

In [ ]:
import pandas as pd
from langchain.evaluation.qa import QAEvalChain

llm = ChatOpenAI(temperature=0.2, model=model_id)
eval_chain = QAEvalChain.from_llm(llm)


Kérdések egyenként

In [ ]:
df = pd.DataFrame(columns=['Question', 'Human Answer', 'AI Answer', 'AI Evaluation'])
for i, question in enumerate(test_questions['Question']):
    memory.clear() # reset memory, hogy ne legyen hatása a következő kérdésre
    ai_ans = chain({"question": question, "chat_history": memory.buffer})
    eval_result = eval_chain({"query": question, "result": ai_ans['answer'], "answer": test_questions['Answer'][i]})
    df.loc[len(df)] = [question, test_questions['Answer'][i], ai_ans['answer'], eval_result['results']]
    
df

In [ ]:
# elmentés
df.to_csv(f"testing/models/openai_k{search_k}_size{chunk_size}_newPrompts.csv", index=False)

Összefüggő beszélgetés

In [ ]:
test_chat = pd.read_csv("testing/test_chat.csv")
print(len(test_chat))
#test_chat

In [ ]:
memory.clear()
chain.verbose=True
df = pd.DataFrame(columns=['Question', 'Human Answer', 'AI Answer', 'AI Evaluation'])
for i, question in enumerate(test_chat['Question']):
    ai_ans = chain({"question": question, "chat_history": memory.buffer})
    eval_result = eval_chain({"query": question, "result": ai_ans['answer'], "answer": test_chat['Answer'][i]})
    df.loc[len(df)] = [question, test_chat['Answer'][i], ai_ans['answer'], eval_result['results']]

df

In [ ]:
# elmentés
df.to_csv(f"testing/models/openai_CHAT_k{search_k}_size{chunk_size}_newPrompts.csv", index=False)

### Rouge score

In [ ]:
%pip install rouge-score==0.1.2 --quiet

In [ ]:
import pandas as pd
from typing import List
from rouge_score import rouge_scorer

def calculate_rouge(df, rouge_types: List[str]):
    scores = []
    scorer = rouge_scorer.RougeScorer(rouge_types, use_stemmer=True)
    for index, row in df.iterrows():
        #Mindegyik sorra az rouge score-t kiszámoljuk, majd a listába tesszük
        score = scorer.score(row["Human Answer"], row["AI Answer"])
        scores.append(score)
    return scores

In [ ]:
# eredmények betöltése
df = pd.read_csv("testing/models/openai_k5_size500.csv")
chat_df = pd.read_csv("testing/models/openai_CHAT_k5_size500.csv")

In [ ]:
# Rouge-2 és rouge-L score kiszámolása, minden sorra
rouge2_scores = calculate_rouge(df, ["rouge2"])
rougeL_scores = calculate_rouge(df, ["rougeL"])

df = df.assign(ROUGE2=rouge2_scores, ROUGEL=rougeL_scores)
df.to_csv("testing/models/openai_k5_size500.csv", index=False)

rouge2_chat_scores = calculate_rouge(chat_df, ["rouge2"])
rougeL_chat_scores = calculate_rouge(chat_df, ["rougeL"])

chat_df = chat_df.assign(ROUGE2=rouge2_chat_scores, ROUGEL=rougeL_chat_scores)
chat_df.to_csv("testing/models/openai_CHAT_k5_size500.csv", index=False)

all_rouge2_scores = rouge2_scores + rouge2_chat_scores
all_rougeL_scores = rougeL_scores + rougeL_chat_scores


In [ ]:
import os
# egy-egy sorozatból átlagszámítás
def calculate_average(scores, rouge_type: str, version: str = ""):
    precisions = [score[rouge_type][0] for score in scores]
    p_avg = sum(precisions) / len(precisions)

    recalls = [score[rouge_type][1] for score in scores]
    r_avg = sum(recalls) / len(recalls)
    
    fmeasures = [score[rouge_type][2] for score in scores]
    f_avg = sum(fmeasures) / len(fmeasures)

    directory = "testing/rouge/"
    if not os.path.exists(directory):
        os.makedirs(directory)

    file_name = f"{directory}average_OpenAI{version}_{rouge_type}.txt"

    with open(file_name, 'w') as f:
        f.write(f"Recall: {r_avg}\n")
        f.write(f"Precision: {p_avg}\n")
        f.write(f"F-measure: {f_avg}\n")

In [ ]:
calculate_average(all_rouge2_scores, "rouge2")
calculate_average(all_rougeL_scores, "rougeL")